In [1]:
import numpy as np
import pandas as pd
import pickle

import math
import json

from daf.datasets import atti_dataset
from daf.utils import dataset_utils
from daf.utils import keras_util

import random
import gensim

/home/fabio/miniconda3/envs/dl/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Recurrent Neural Networks

In [2]:
(x_train, y_train), (x_test, y_test) = atti_dataset.load_data(num_words=None)
label_index_dict = atti_dataset.get_label_index()
id_word_dict = atti_dataset.get_word_index()

num_words = max([max(x) for x in x_train]) + 1
num_words

52396

In [3]:
x_train_text = [[id_word_dict[str(x)] for x in seq] for seq in x_train]

In [20]:
model = gensim.models.Word2Vec(x_train_text, workers=5, min_count=1, iter=10)

In [23]:
model.most_similar('art')

/home/fabio/miniconda3/envs/dl/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('articolo', 0.8189318180084229),
 ('bis', 0.5358280539512634),
 ('artt', 0.5259966254234314),
 ('comma', 0.4990021586418152),
 ('lettera', 0.47949644923210144),
 ('lett', 0.47568076848983765),
 ('arat', 0.4680165648460388),
 ('articoli', 0.46294692158699036),
 ('minist', 0.44396334886550903),
 ('quater', 0.44160643219947815)]

In [22]:
model.save('../data/dataset/atti.word2vec')

In [18]:
m = gensim.models.Word2Vec.load('../data/dataset/atti.word2vec')